In [1]:
import os
os.chdir('../../..')

In [2]:
from convokit import Corpus, download

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [4]:
corpus.print_summary_stats()

Number of Users: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Bag-of-words prediction for utterances having positive scores

In [5]:
from convokit import BoWClassifier, BoWTransformer

In [6]:
bow_transformer = BoWTransformer(obj_type="utterance")

Initializing default unigram CountVectorizer...


In [7]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'user': User('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': ''})


In [8]:
bow_transformer.fit_transform(corpus)

In [9]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'user': User('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': '', 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>})


In [10]:
bow_classifer = BoWClassifier(obj_type="utterance", labeller=lambda utt: utt.meta['score'] > 0)

Initializing default classification model (standard scaled logistic regression)


In [11]:
bow_classifer.fit(corpus)

In [12]:
bow_classifer.transform(corpus)

In [13]:
next(corpus.iter_utterances()).meta

{'score': 2,
 'top_level_comment': None,
 'retrieved_on': -1,
 'gilded': -1,
 'gildings': None,
 'subreddit': 'Cornell',
 'stickied': False,
 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/',
 'author_flair_text': '',
 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 42 stored elements in Compressed Sparse Row format>,
 'prediction': True,
 'pred_score': 0.9999998491105527}

In [14]:
next(corpus.iter_utterances()).meta['bow_vector'].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [15]:
bow_classifer.summarize(corpus).head()

,prediction,pred_score
id,,
dhhm9sa,True,1.0
dw553ml,True,1.0
dvzmhdx,True,1.0
dvzpp79,True,1.0
dw0imao,True,1.0


In [16]:
corpus.get_utterance('15enm8').text

'One, just to get this out of the way: I\'m only a sophomore in high school. In spite of this, my high school is one of the top public schools in New Jersey (and to put it bluntly it\'s a very affluent area... although I\'m not necessarily affluent like my classmates). The point of telling you guys that is kids start talking about all these amazing schools they want to go to in like eighth grade, so I know quite a bit about colleges. As stated in the title, I really want to go to Cornell, and I just was hoping that some of you guys and girls on here would be awesome enough to give out some SAT scores, ACT scores (if you took them), and extra curricular activities you guys got/did? My unweighted GPA is a 3.8 (weighted is a 4.2), and my first PSAT was an overall 1900, and from taking that I (not to sound cocky here) *know* that I\'m going to get that score up a *lot*. I\'m in all the highest level classes I can be in, and I\'m looking to take multiple AP courses next year (junior). Do yo

In [17]:
bow_classifer.get_coefs(feature_names=bow_transformer.get_vocabulary()).head()

,coef
feat_name,
child,1.270001
appealing,1.115690
whereas,1.109702
messed,1.081836
focusing,1.071464


In [18]:
bow_classifer.get_coefs(feature_names=bow_transformer.get_vocabulary()).tail()

,coef
feat_name,
criteria,-0.667785
entered,-0.722992
bathroom,-0.742234
korea,-0.764383
goal,-0.784593


In [19]:
from convokit import Classifier

In [20]:
c = Classifier(pred_feats = None, obj_type = "utterance", labeller = lambda utt: utt.meta['score'] > 0)

In [21]:
y_true, y_pred = c.get_y_true_pred(corpus)

In [22]:
c.base_accuracy(corpus)

0.9279546644822538

In [23]:
c.accuracy(corpus)

0.9491452589737737

In [24]:
print(c.classification_report(corpus))

              precision    recall  f1-score   support

       False       0.88      0.34      0.49      5365
        True       0.95      1.00      0.97     69102

    accuracy                           0.95     74467
   macro avg       0.91      0.67      0.73     74467
weighted avg       0.95      0.95      0.94     74467



## Bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [25]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [26]:
corpus.print_summary_stats()

Number of Users: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [27]:
len(top_level_comment_ids)

32893

In [28]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids)


['c3od15i', 'c3p8bze', 'c3oyf4d', 'c3p1rn8', 'c3ocsyl']


In [29]:
threads_corpus.print_summary_stats()

Number of Users: 6160
Number of Utterances: 63697
Number of Conversations: 32888


In [30]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [31]:
bow_transformer2 = BoWTransformer(obj_type="conversation", 
                    text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                    selector=lambda convo: convo.meta['thread_doubles'] is not None,
                   )

Initializing default unigram CountVectorizer...


In [32]:
bow_transformer2.fit_transform(threads_corpus)

In [34]:
bow_classifier2 = BoWClassifier(obj_type="conversation", 
                    selector=lambda convo: convo.meta['thread_doubles'] is not None,
                    labeller=lambda convo: convo.meta['thread_doubles']
                   )

Initializing default classification model (standard scaled logistic regression)


In [35]:
bow_classifier2.fit_transform(threads_corpus)

In [36]:
bow_classifier2.summarize(threads_corpus).head()

,prediction,pred_score
id,,
dt05qyf,True,1.0
dandio0,True,1.0
dwa6k96,True,1.0
dsldpxg,True,1.0
e70wjy3,True,1.0


In [37]:
bow_classifier2.summarize(threads_corpus).tail()

,prediction,pred_score
id,,
drduxx1,False,2.464221e-12
dl7q7n2,False,8.164543e-14
dxfib8r,False,2.716435e-15
dwqaa06,False,2.681522e-16
d8y9akn,False,1.604632e-16


In [48]:
bow_classifier2.base_accuracy(threads_corpus)

0.6761904761904762

In [49]:
bow_classifier2.accuracy(threads_corpus)

0.9992063492063492

In [50]:
print(bow_classifier2.classification_report(threads_corpus))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       852
        True       1.00      1.00      1.00       408

    accuracy                           1.00      1260
   macro avg       1.00      1.00      1.00      1260
weighted avg       1.00      1.00      1.00      1260



In [42]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary())

,coef
feat_name,
finally,0.611794
ya,0.513358
open,0.508405
conservative,0.481661
html,0.463145
nobody,0.456153
knows,0.441402
yet,0.417242
things,0.414036


In [46]:
bow_classifier2.confusion_matrix(threads_corpus)

array([[852,   0],
       [  1, 407]])

In [47]:
bow_classifier2.base_accuracy(threads_corpus)

0.6761904761904762